## Manual Pipeline

In the previous notebook `0_simulation.ipynb` we simulated dataset. Our simulated dataset includes __detections__ and __resection_mni__. Now we are ready to run `Manual detections pipeline` using __detections__ as manually selected spikes and compare the outlut to the __resection_mni__ (ground truth). In this notebook we have the following goals:
1. Prepare and run irritative zone delineation using __detections__ timepoints.
2. Evaluate the quality of the irritative zone prediction using __resection__.

We will use the results presented in this nodebook as a baseline for the automated pipelines.

NOTE: To import the megspikes modules we should change the working directory if we are running this example from the cloned GitHub repository.

In [ ]:
import os
from pathlib import Path

import matplotlib.pylab as plt
import mne
import numpy as np

# change to the root directory of the project
if os.getcwd().split("/")[-1] == "examples":
    os.chdir('..')

from megspikes.simulation.simulation import Simulation

# Setup the path for the simulation
sample_path = Path(os.getcwd()) / 'examples' / 'data' / '1_manual_pipeline'
sample_path.mkdir(exist_ok=True, parents=True)

%load_ext autoreload
%autoreload 2

## Prepare the simulation

In [ ]:
sim = Simulation(sample_path, n_events=[15, 15, 0, 0])
sim.simulate_dataset()
sim

`sim.detections` is an array of spike peak localizations in samples. We will use it as manual detections.

In [ ]:
sim.detections

`sim.clusters` is an array of clusters that corresponds to the simulation source index.

In [ ]:
sim.clusters

## Run Manual Pipeline

In [ ]:
from megspikes.pipeline import iz_prediction_pipeline
from sklearn import set_config
set_config(display='diagram')

In [ ]:
params = {
    'PrepareClustersDataset': {'detection_sfreq': 1000.}
}
pipe = iz_prediction_pipeline(sim.case_manager, params)
pipe

In [ ]:
detections = {
    'spikes': sim.detections,
    'clusters': sim.clusters - 1
}
dataset, meg_data = pipe.fit_transform((detections, sim.raw_simulation.copy()))

`dataset` is an instance of xarray.Dataset, and it includes all the results. We can explore the results and the corresponding metadata.

In [ ]:
dataset

## View localized cluster

First we should convert numpy ndarray to mne.SourceEstimate. We use `array_to_stc` for this.

In [ ]:
from megspikes.localization.localization import array_to_stc

stc = array_to_stc(dataset.mne_localization.values[0, 0, :, :],
                   sim.case_manager.fwd['ico5'],
                   sim.case_manager.case)

Now we can plot the cluster.

In [ ]:
%matplotlib qt5
brain = stc.plot(subjects_dir=sim.case_manager.freesurfer_dir, hemi='both')

## View clusters using Cluster Slope Viewer

In [ ]:
%matplotlib qt5
import mne
from megspikes.visualization.visualization import ClusterSlopeViewer
mne.viz.set_3d_backend('pyvista')

In [ ]:
pc = ClusterSlopeViewer(dataset, sim.case_manager)

In [ ]:
pc.view()

Let's load the updated dataset to check it.

In [ ]:
import xarray as xr
updated_ds = xr.load_dataset(pc.fname_save_ds)

## Veiw IZ prediction

In [ ]:
from megspikes.localization.localization import array_to_stc
stc = array_to_stc(dataset.iz_prediction.values[:, -1],
                   sim.case_manager.fwd['ico5'],
                   sim.case_manager.case)

In [ ]:
%matplotlib qt5

# Add function from visualization 
brain = stc.plot(subjects_dir=sim.case_manager.freesurfer_dir, hemi='both')

## Evaluate Irritative Zone prediction

In [ ]:
from megspikes.scoring.scoring import ScoreIZPrediction
scorer = ScoreIZPrediction()

In [ ]:
scorer.score(dataset, sim.mni_resection, 'peak')

### Plot resection and prediction

In [ ]:
%matplotlib inline
from nilearn import plotting
fig, ax = plt.subplots(figsize=(15, 7))

display = plotting.plot_glass_brain(
            None, display_mode='lzry', figure=fig, axes=ax)

display.add_markers(scorer.detection_mni, marker_color='tomato', alpha=0.2)
display.add_markers(sim.mni_resection, marker_color='indigo', alpha=0.6)


## Test noise levels

In [ ]:
params = {
    'PrepareClustersDataset': {'detection_sfreq': 1000.}
}
detections = {
    'spikes': sim.detections,
    'clusters': sim.clusters - 1
}

for noise in [0.5, 1, 2, 5, 10]:
    sim.simulate_dataset(noise_scaler=noise)
    pipe = iz_prediction_pipeline(sim.case_manager, params)
    dataset, _ = pipe.fit_transform((detections, sim.raw_simulation.copy()))
    baseline_score = scorer.score(dataset, sim.mni_resection, 'baseline')
    slope_score = scorer.score(dataset, sim.mni_resection, 'slope')
    peak_score = scorer.score(dataset, sim.mni_resection, 'peak')
    print(f"Scores: baseline={baseline_score}, slope={slope_score}, peak={peak_score}")